In [1]:
#Importing important modules
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score

Using TensorFlow backend.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Reading Data

In [0]:
import h5py
import numpy as np

h5f = h5py.File('/content/drive/My Drive/CNN/SVHN_single_grey1.h5', 'r')

# Load the training, test and validation set
x_train = h5f['X_train'][:]
y_train = h5f['y_train'][:]
x_test = h5f['X_test'][:]
y_test = h5f['y_test'][:]


# Close this file
h5f.close()

In [4]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(42000, 32, 32)
(42000,)
(18000, 32, 32)
(18000,)


In [5]:
print(y_train.shape)
print(y_test.shape)

(42000,)
(18000,)


In [0]:
# we will convert the (32*32) vector into single dimensional vector of 1 * 1024
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1]*x_train.shape[2]) 
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1]*x_test.shape[2]) 

In [0]:
from sklearn.preprocessing import StandardScaler
std=StandardScaler()
x_train=std.fit_transform(x_train)
x_test=std.transform(x_test)

In [8]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(42000, 1024)
(42000,)
(18000, 1024)
(18000,)


## KNN Classifier


In [0]:
k = [50,100,150,200]
knn = KNeighborsClassifier(weights = 'uniform', metric='euclidean')
parameters = {'n_neighbors' : k}
GS = GridSearchCV(knn, parameters, scoring = 'accuracy', cv=3)

In [10]:
GS.fit(x_train, y_train)

GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                            metric='euclidean',
                                            metric_params=None, n_jobs=None,
                                            n_neighbors=5, p=2,
                                            weights='uniform'),
             iid='warn', n_jobs=None,
             param_grid={'n_neighbors': [50, 100, 150, 200]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='accuracy', verbose=0)

In [11]:
GS.cv_results_ 

{'mean_fit_time': array([2.23427439, 2.23570379, 2.22843027, 2.24551511]),
 'mean_score_time': array([707.61600455, 699.49113623, 700.42179616, 696.0505627 ]),
 'mean_test_score': array([0.50788095, 0.50180952, 0.49228571, 0.48447619]),
 'param_n_neighbors': masked_array(data=[50, 100, 150, 200],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'n_neighbors': 50},
  {'n_neighbors': 100},
  {'n_neighbors': 150},
  {'n_neighbors': 200}],
 'rank_test_score': array([1, 2, 3, 4], dtype=int32),
 'split0_test_score': array([0.51438977, 0.5102478 , 0.49810755, 0.49260873]),
 'split1_test_score': array([0.50157143, 0.4955    , 0.48592857, 0.47678571]),
 'split2_test_score': array([0.50768022, 0.4996785 , 0.49281989, 0.48403229]),
 'std_fit_time': array([0.0263286 , 0.01340487, 0.0243888 , 0.01023614]),
 'std_score_time': array([3.18184178, 2.03813204, 2.43357327, 9.8073048 ]),
 'std_test_score': array([0.00523527, 0.00620672, 0.00

In [12]:
GS.best_estimator_

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='euclidean',
                     metric_params=None, n_jobs=None, n_neighbors=50, p=2,
                     weights='uniform')

In [13]:
model=GS.best_estimator_
model.fit(x_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='euclidean',
                     metric_params=None, n_jobs=None, n_neighbors=50, p=2,
                     weights='uniform')

In [0]:
y_pred=model.predict(x_test)

In [15]:
accuracy_score(y_test,y_pred)

0.5478333333333333

In [16]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

array([[1295,   53,   30,   40,   50,   32,   85,   35,   74,  120],
       [ 105, 1362,   45,   70,   80,   27,   33,   59,   21,   26],
       [ 101,  222,  976,   87,   53,   39,   29,  156,   51,   89],
       [ 107,  280,   84,  761,   58,  162,   32,   65,   94,   76],
       [ 102,  256,   35,   50, 1206,   18,   39,   23,   29,   54],
       [ 163,  179,   40,  217,   70,  735,  135,   31,  116,   82],
       [ 318,  124,   31,   50,  138,  103,  785,   23,  203,   57],
       [ 103,  193,  103,   69,   30,   37,   42, 1158,   28,   45],
       [ 259,  137,   49,   90,   93,   91,  221,   19,  730,  123],
       [ 295,  161,   50,   67,   68,   81,   44,   82,  103,  853]])

# Neural Network (RELU Activation with 2 hidden layers and Batch Normalization)


In [0]:
from keras.layers.normalization import BatchNormalization
from sklearn.metrics import accuracy_score
from keras.models import Sequential 
from keras.layers import Dense, Activation 
from keras.initializers import he_normal

In [0]:
import keras
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

In [19]:
model_batch = Sequential()
model_batch.add(Dense(128, activation='relu',input_shape=(1024,),kernel_initializer=he_normal(seed=None)))
model_batch.add(BatchNormalization())
model_batch.add(Dense(64, activation='relu',kernel_initializer=he_normal(seed=None)) )
model_batch.add(BatchNormalization())
model_batch.add(Dense(10, activation='softmax'))
model_batch.summary()






Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 128)               131200    
_________________________________________________________________
batch_normalization_1 (Batch (None, 128)               512       
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
batch_normalization_2 (Batch (None, 64)                256       
_________________________________________________________________
dense_3 (Dense)              (None, 10)                650       
Total params: 140,874
Trainable params: 140,490
Non-trainable params: 384
_________________________________________________________________


In [20]:
model_batch.compile(optimizer='SGD', loss='categorical_crossentropy', metrics=['accuracy'])

history12 = model_batch.fit(x_train, y_train, 
                           batch_size=128,
                           epochs=50, verbose=1, 
                           validation_data=(x_test, y_test))



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 42000 samples, validate on 18000 samples
Epoch 1/50





42000/42000 [==============================] - 4s 94us/step - loss: 2.0027 - acc: 0.3183 - val_loss: 1.6798 - val_acc: 0.4497
Epoch 2/50
42000/42000 [==============================] - 3s 63us/step - loss: 1.4438 - acc: 0.5468 - val_loss: 1.3264 - val_acc: 0.5927
Epoch 3/50
42000/42000 [==============================] - 3s 65us/step - loss: 1.2059 - acc: 0.6377 - val_loss: 1.1644 - val_acc: 0.6421
Epoch 4/50
42000/42000 [==============================] - 3s 62us/step - loss: 1.0678 - acc: 0.6820 - val_loss: 1.1074 - val_acc: 0.6628
Epoch 5/50
42000/42000 [==============================] - 3s 63us/step - loss: 0.9761 - acc: 0.7079 - val_loss: 1.0282 - val_acc: 0.6846
Epoch 6/50
42000/42000 [==============================] - 3s 64us/step - loss: 0.9073 - acc: 0.7273 - val_loss: 1.0503 - val_acc: 0.6741
Epoch 7/50
42000/42000 [

In [21]:
max(history12.history["val_acc"])

0.8233888889948527

In [23]:
# Initializing prettytable
from prettytable import PrettyTable
Model_Performance = PrettyTable()
# Adding columns
Model_Performance.add_column("INDEX.",[1,2])
Model_Performance.add_column("MODEL_NAME",["KNN","Neural Net Model"])
Model_Performance.add_column("TESTING ACCURACY",[54.7,82.3])

# Printing the Model_Performance
print(Model_Performance)

+--------+------------------+------------------+
| INDEX. |    MODEL_NAME    | TESTING ACCURACY |
+--------+------------------+------------------+
|   1    |       KNN        |       54.7       |
|   2    | Neural Net Model |       82.3       |
+--------+------------------+------------------+
